# Markov Chain Monte Carlo (MCMC) Sampling with the Metropolis Algorithm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import ProgressBar

In [2]:
def random_state(shape):
    return np.random.randint(0,2,shape)*2 - 1

## Outline of Algorithm (Metropolis Step):

0. Start with a random state (1).
1. Consider a random change to the state. In particular, choose one spin at random and consider flipping it. The new candidate state is state (2).
2. Compute the ratio of the probabilities $r=P_2/P_1$. For a thermal system, this is the relative Boltzmann factor $\exp(-(E_2-E_1)/kT)$.
3. If $r \ge 1$, accept the change. If $r < 1$, accept the change with probability $r$. If the change is rejected, state (1) is sampled a second time.
4. Repeat from step 1.

Over a long period of time, the states will be sampled proportionally to their probabilities (Boltzmann factors). In practice, an initial number of Metropolis steps are performed without retaining the results; this is the ''burn in'' phase. Then, between each sample, many Metropolis steps are performed in order to make the samples statistically independent.